In [6]:
import logging
import os
import concurrent.futures
import importlib
from pathlib import Path
import re
import subprocess
import sys
import time

import colorama
import jsonlines
import matplotlib.pyplot as plt
import numpy as np
import rich
import tqdm

In [1]:
##########################################################################################
# Prepare the file lists
##########################################################################################
def parse_num(path):
    numbers = re.findall("[0-9]+", str(Path(path).name))
    assert len(numbers) == 1, len(numbers)
    return int(numbers[0])

target_dir = Path("iterated_decoding_output/first_test/").resolve()
wd_dir_retr = target_dir / "transformed_retr"
wd_dir_gen = target_dir / "transformed_gen"

contexts_outputs = sorted(target_dir.glob("contexts_*"), key=parse_num)
reader_outputs = sorted(target_dir.glob("generated_*"), key=parse_num)

print(f"{len(contexts_outputs) = }")
print(f"{len(reader_outputs) = }")

NameError: name 'Path' is not defined

In [ ]:
def count_lines(path):
    return int(subprocess.check_output(
        ["wc", "-l", str(path)]
    ).decode().strip().split()[0])


def job(packed):
    i, path = packed
    assert i == parse_num(path), (i, parse_num(path))
    loop = []

    with jsonlines.open(path) as fin:
        for line in fin:
            loop.extend(line)


reader_gens = list(map(job, enumerate(reader_outputs)))

[8757, 8757, 8757, 8757, 8757, 8757, 8757, 8757, 8757, 8757, 8757, 8757, 8757, 8757, 8757]


In [12]:
##########################################################################################
# Clean and move the data
##########################################################################################

orig_targets = Path("../GAR/data/nq-answer/val.target").resolve()
wd_targets = wd_dir_gen / "val_targets.txt"

# if not wd_targets.exists():
#     shutil.copy(orig_targets, wd_targets)

    
# for i, lines in enumerate(tqdm.tqdm(reader_outputs)):
#     with open(wd_dir_gen/f"val_predictions-{i}.txt", "w") as fout:
#         for line in lines:
#             line = line.replace("</s>", "").replace("<pad>", "").strip()
#             fout.write(line + "\n")
            
contexts_outputs_wd = sorted(wd_dir_retr.glob("contexts_*"), key=parse_num)
reader_outputs_wd = sorted(wd_dir_gen.glob("val_predictions-*"), key=parse_num)

print(f"{len(contexts_outputs_wd) = }")
print(f"{len(reader_outputs_wd) = }")

len(contexts_outputs_wd) = 15
len(reader_outputs_wd) = 15


In [ ]:
# !which python
# !python compute_rouge.py $reader_outputs_wd

/home/mila/g/gagnonju/condaless/bin/python
/home/mila/g/gagnonju/IteratedDecoding/jobs/iterated_decoding_output/first_test/transformed
All done!
Total time: 0.2715453691780567


In [19]:
paths_rouge = sorted(wd_dir_gen.glob("rouge*"), key=parse_num)
print(f"{len(paths_rouge) = }")

len(paths_rouge) = 15


In [56]:
def read_file(path: Path):
    with open(path) as fin:
        return fin.read()

def read_jsonl(path: Path):
    assert str(path).endswith(".jsonl"), path
    chars = read_file(path)
    with jsonlines.open(path) as fin:
        return list(fin)


In [68]:
f0_path = contexts_outputs_wd[0]
f0 = read_jsonl(f0_path)

for path_f1 in sorted(set(contexts_outputs_wd) - {f0_path}, key=parse_num):
    f1 = read_jsonl(path_f1)
    assert len(f0) == len(f1), (len(f0), len(f1))
    qty_all = len(f0[0])

    goods = 0
    bads = 0
    for f0_l, f1_l in zip(f0, f1):   
        for index in range(len(f0_l["ids"])):
            set_0 = {int(x) for x in f0_l["ids"][index]}
            set_1 = {int(x) for x in f1_l["ids"][index]}
            set_or = set_0 | set_1
            set_and = set_0 & set_1
            ratio = len(set_and) / len(set_or)
            if len(set_and) != 0:
                goods += 1
            else:
                bads += 1
    print(f"{parse_num(path_f1)}: {goods / (goods + bads):.2%} that have more than absolute 0 overlap")


1: 1.14% that have more than absolute 0 overlap
2: 0.66% that have more than absolute 0 overlap
3: 0.55% that have more than absolute 0 overlap
4: 0.59% that have more than absolute 0 overlap
5: 0.53% that have more than absolute 0 overlap
6: 0.53% that have more than absolute 0 overlap
7: 0.42% that have more than absolute 0 overlap
8: 0.39% that have more than absolute 0 overlap
9: 0.57% that have more than absolute 0 overlap
10: 0.46% that have more than absolute 0 overlap
11: 0.37% that have more than absolute 0 overlap
12: 0.31% that have more than absolute 0 overlap
13: 0.46% that have more than absolute 0 overlap
14: 0.49% that have more than absolute 0 overlap
